<a href="https://colab.research.google.com/github/pachterlab/seqspec/blob/main/examples/seqspec_dev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -rf seqspec
!git clone https://github.com/pachterlab/seqspec.git
!cd seqspec && git checkout libspec && pip install --quiet .

Cloning into 'seqspec'...
remote: Enumerating objects: 1400, done.
remote: Counting objects: 100% (578/578), done.
remote: Compressing objects: 100% (294/294), done.
remote: Total 1400 (delta 365), reused 457 (delta 280), pack-reused 822
Receiving objects: 100% (1400/1400), 5.66 MiB | 10.59 MiB/s, done.
Resolving deltas: 100% (886/886), done.
Branch 'libspec' set up to track remote branch 'libspec' from 'origin'.
Switched to a new branch 'libspec'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.0 MB/s eta 0:00:00


In [3]:
from seqspec.utils import load_spec

In [8]:
spec.sequence_spec

[{'read_id': 'R1.fastq.gz', 'read_name': 'Read 1', 'modality': 'rna', 'primer_id': 'truseq_read1', 'min_len': 28, 'max_len': 28, 'strand': 'pos'},
 {'read_id': 'R2.fastq.gz', 'read_name': 'Read 2', 'modality': 'rna', 'primer_id': 'truseq_read2', 'min_len': 1, 'max_len': 98, 'strand': 'pos'},
 {'read_id': 'I1.fastq.gz', 'read_name': 'Index 1', 'modality': 'rna', 'primer_id': 'truseq_read2', 'min_len': 8, 'max_len': 8, 'strand': 'neg'}]

In [23]:
def get_cuts(regions, cuts=[]):
    if not cuts:
        cuts = []
    prev = 0
    for r in regions:
        nxt = prev + r.max_len
        cuts.append((prev, nxt))
        prev = nxt
    return cuts

In [29]:
from collections import defaultdict

In [44]:
# testing out index functionality
spec_fn = "seqspec/lib_test.yaml"

reads = ["R1.fastq.gz"]
spec = load_spec(spec_fn)
modality = "rna"
read_id = reads[0]
rev = False

## Start function
index = defaultdict()

# get all atomic elements from library
leaves = spec.get_modality(modality).get_leaves()

# get the read object and primer id
read = [i for i in spec.sequence_spec if i.read_id == read_id][0]
primer_id = read.primer_id

# get the index of the primer in the list of leaves (ASSUMPTION, 5'->3' and primer is an atomic element)
primer_idx = [i for i, l in enumerate(leaves) if l.region_id == primer_id][0]

# If we are on the opposite strand, we go in the opposite way
if rev:
    rgn = leaves[:primer_idx][::-1]
else:
    rgn = leaves[primer_idx + 1:]

# get the cuts for all of the atomic elements (tuples of 0-indexed start stop)
cuts = get_cuts(rgn)

# associate each cut with its region type
for idx, r in enumerate(rgn):
    t = r.region_type
    c = cuts[idx]
    index[c] = t

In [45]:
# Now that we have the cuts, we have to do an interval intersection
# given the interval of the read (0, max_cycles)
# return the intersection with the indexed elements

In [74]:
index = {(0, 16): 'barcode',
             (16, 27): 'umi',
             (28, 29): 'umi2',
             (29, 126): 'cdna',
             (126, 160): 'truseq_read2',
             (160, 168): 'index7',
             (168, 192): 'illumina_p7'}

In [81]:
# get max cycles from read by read id
read_itv = (0, read.max_len)
read_stop = read_itv[1]
new_index = defaultdict()

# three cases
for (itv_start, itv_stop), e in index.items():

  # A: read is shorter than itv_start of next element
  if read_stop < itv_start:
    continue

  # B: read splits an element (same as case c)
  elif read_stop >= itv_start:
    new_index[itv_start, min(itv_stop, read_stop)] = e

In [82]:
read_stop

28

In [83]:
new_index

defaultdict(None, {(0, 16): 'barcode', (16, 22): 'umi', (28, 28): 'cdna'})

In [54]:
new_indices

defaultdict(None, {(0, 16): 'barcode', (16, 28): 'umi'})

In [49]:
# goal: to return indices such that they are limited by the max len of the fastq

defaultdict(None,
            {(0, 16): 'barcode',
             (16, 28): 'umi',
             (28, 126): 'cdna',
             (126, 160): 'truseq_read2',
             (160, 168): 'index7',
             (168, 192): 'illumina_p7'})